In [2]:
import pandas as pd
import numpy as np
import math
import logging
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from decimal import Decimal
from matplotlib.colors import BoundaryNorm

from statannotations.Annotator import Annotator

from scipy import stats
import statsmodels.stats.multitest
from decimal import Decimal

from sklearn import preprocessing

import pickle 
from matplotlib import rcParams

import warnings
warnings.filterwarnings('ignore')

import utils_statistics, utils_plotting, utils_misc

# 1. Data Loading

In [23]:
dystonia_genetics = pd.read_csv('dataset/genetic_dystonia_pallidal_neuron_activity.csv')
dystonia_genetics = dystonia_genetics[dystonia_genetics.patient!="patient1"] # removing benign-SGCE case

# these two features can have infinite values due to 0 division problem. while their measurement
dystonia_genetics['pause_index'].replace(np.inf,dystonia_genetics.loc[dystonia_genetics['pause_index'] != np.inf, 'pause_index'].max(),inplace=True)
dystonia_genetics['pause_ratio'].replace(np.inf,dystonia_genetics.loc[dystonia_genetics['pause_ratio'] != np.inf, 'pause_ratio'].max(),inplace=True)

genes             = ["AOPEP","GNAL","KMT2B","PANK2","PLA2G6","SGCE","THAP1","TOR1A","VPS16"]

# 2. Mean + Std Values of Features

In [44]:
for feature in ['firing_rate', 'firing_regularity', 'cv', 'lv', 'isi_mean', 'isi_std', 'isi_skewness', 'isi_rho', 'asymmetry_index', 'pause_index', 'pause_ratio',
                'pause_duration', 'pause_freq', 'pause_count', 'pause_spike_proportion', 'pause_time_proportion']:
    print(feature + "-------------------------------")
    for gene in genes:
        # to represent in ms instead of seconds (ISI)
        if(feature in ["isi_mean","isi_std"]): 
            gene_mean = dystonia_genetics[(dystonia_genetics.gene == gene)][feature].mean() * 1000
            gene_std  = dystonia_genetics[(dystonia_genetics.gene == gene)][feature].std() * 1000
        # to represent the proportions as 100% percentage
        elif(feature in ['pause_time_proportion', 'pause_spike_proportion']):
            gene_mean = dystonia_genetics[(dystonia_genetics.gene == gene)][feature].mean() * 100
            gene_std  = dystonia_genetics[(dystonia_genetics.gene == gene)][feature].std() * 100
        else:
            gene_mean = dystonia_genetics[(dystonia_genetics.gene == gene)][feature].mean()
            gene_std  = dystonia_genetics[(dystonia_genetics.gene == gene)][feature].std()
        print(gene + " : " + str(round(gene_mean, 2)) + "±" + str(round(gene_std, 2)))
    

firing_rate-------------------------------
AOPEP : 15.35±6.9
GNAL : 13.41±9.73
KMT2B : 16.71±21.76
PANK2 : 20.3±26.36
PLA2G6 : 16.73±10.59
SGCE : 14.95±11.96
THAP1 : 23.96±17.74
TOR1A : 20.21±15.46
VPS16 : 17.15±10.99
firing_regularity-------------------------------
AOPEP : 0.23±0.32
GNAL : -0.13±0.46
KMT2B : -0.02±0.43
PANK2 : 0.32±0.49
PLA2G6 : -0.02±0.49
SGCE : -0.14±0.36
THAP1 : 0.33±0.4
TOR1A : 0.06±0.4
VPS16 : 0.06±0.47
cv-------------------------------
AOPEP : 1.14±0.34
GNAL : 1.82±0.8
KMT2B : 1.6±1.1
PANK2 : 1.18±0.64
PLA2G6 : 1.59±0.77
SGCE : 1.77±1.07
THAP1 : 1.45±1.12
TOR1A : 1.49±0.85
VPS16 : 1.51±0.98
lv-------------------------------
AOPEP : 0.72±0.15
GNAL : 0.79±0.24
KMT2B : 0.81±0.25
PANK2 : 0.69±0.23
PLA2G6 : 0.75±0.22
SGCE : 0.82±0.23
THAP1 : 0.62±0.16
TOR1A : 0.78±0.22
VPS16 : 0.77±0.19
isi_mean-------------------------------
AOPEP : 78.08±31.98
GNAL : 116.25±77.52
KMT2B : 117.18±91.53
PANK2 : 91.23±68.5
PLA2G6 : 89.38±66.0
SGCE : 96.52±58.25
THAP1 : 62.93±44.84
TOR1

In [48]:
for feature in ['burst_index', 'burst_duration', 'burst_freq', 'inter_burst_duration', 'burst_count', 'burst_average_spikes', 'burst_spike_proportion']:
    print(feature + "-------------------------------")
    for gene in genes:
        # to represent the proportions as 100% percentage
        if(feature in ['burst_spike_proportion']):
            gene_mean = dystonia_genetics[(dystonia_genetics.gene == gene) & (dystonia_genetics.is_bursting == 1)][feature].mean() * 100
            gene_std  = dystonia_genetics[(dystonia_genetics.gene == gene) & (dystonia_genetics.is_bursting == 1)][feature].std() * 100
        else:
            gene_mean = dystonia_genetics[(dystonia_genetics.gene == gene) & (dystonia_genetics.is_bursting == 1)][feature].mean()
            gene_std  = dystonia_genetics[(dystonia_genetics.gene == gene) & (dystonia_genetics.is_bursting == 1)][feature].std()
        print(gene + " : " + str(round(gene_mean, 2)) + "±" + str(round(gene_std, 2)))
    

burst_index-------------------------------
AOPEP : 20.63±15.63
GNAL : 31.6±26.26
KMT2B : 24.96±26.92
PANK2 : 25.09±34.62
PLA2G6 : 18.29±25.71
SGCE : 24.05±25.68
THAP1 : 8.34±5.93
TOR1A : 28.82±28.41
VPS16 : 22.42±19.42
burst_duration-------------------------------
AOPEP : 0.11±0.1
GNAL : 0.2±0.18
KMT2B : 0.17±0.25
PANK2 : 0.29±0.29
PLA2G6 : 0.12±0.12
SGCE : 0.13±0.18
THAP1 : 0.17±0.24
TOR1A : 0.21±0.35
VPS16 : 0.13±0.17
burst_freq-------------------------------
AOPEP : 208.1±103.23
GNAL : 116.8±60.91
KMT2B : 130.3±83.52
PANK2 : 100.28±88.02
PLA2G6 : 141.29±71.41
SGCE : 142.58±76.78
THAP1 : 144.22±95.62
TOR1A : 137.48±80.32
VPS16 : 151.7±80.46
inter_burst_duration-------------------------------
AOPEP : 3.01±0.49
GNAL : 3.89±2.07
KMT2B : 4.58±3.61
PANK2 : 3.11±1.4
PLA2G6 : 3.05±1.62
SGCE : 3.52±1.84
THAP1 : 2.14±0.73
TOR1A : 3.28±1.67
VPS16 : 3.12±1.7
burst_count-------------------------------
AOPEP : 1.33±0.58
GNAL : 4.48±4.55
KMT2B : 3.85±3.79
PANK2 : 3.0±1.78
PLA2G6 : 5.28±4.52
SGCE :

In [52]:
for feature in ['oscillation_frequency_delta_band', 'oscillation_frequency_theta_band', 'oscillation_frequency_alpha_band', 
                'oscillation_frequency_beta_band','oscillation_frequency_gamma_band']:
    print(feature + "-------------------------------")
    for gene in genes:
        gene_mean = dystonia_genetics[(dystonia_genetics.gene == gene) & (dystonia_genetics.is_oscillatory == 1)][feature].mean()
        gene_std  = dystonia_genetics[(dystonia_genetics.gene == gene) & (dystonia_genetics.is_oscillatory == 1)][feature].std()
        print(gene + " : " + str(round(gene_mean, 2)) + "±" + str(round(gene_std, 2)))
    

oscillation_frequency_delta_band-------------------------------
AOPEP : 0.92±0.0
GNAL : 1.25±0.71
KMT2B : 1.01±0.11
PANK2 : 0.93±0.05
PLA2G6 : 1.04±0.28
SGCE : 1.04±0.32
THAP1 : 0.92±0.0
TOR1A : 1.0±0.35
VPS16 : 0.96±0.25
oscillation_frequency_theta_band-------------------------------
AOPEP : 3.66±0.0
GNAL : 4.87±1.44
KMT2B : 4.21±1.02
PANK2 : 3.83±0.55
PLA2G6 : 4.46±1.16
SGCE : 4.42±1.19
THAP1 : 3.8±0.51
TOR1A : 4.12±0.9
VPS16 : 4.19±0.95
oscillation_frequency_alpha_band-------------------------------
AOPEP : 8.79±1.66
GNAL : 8.33±1.1
KMT2B : 8.76±1.34
PANK2 : 8.51±1.61
PLA2G6 : 9.11±1.38
SGCE : 8.86±1.26
THAP1 : 8.63±1.52
TOR1A : 8.66±1.33
VPS16 : 8.67±1.33
oscillation_frequency_beta_band-------------------------------
AOPEP : 14.65±5.07
GNAL : 16.48±4.48
KMT2B : 16.74±5.56
PANK2 : 20.4±5.02
PLA2G6 : 17.51±4.82
SGCE : 18.82±5.02
THAP1 : 15.14±3.79
TOR1A : 17.23±5.84
VPS16 : 17.8±5.37
oscillation_frequency_gamma_band-------------------------------
AOPEP : 52.25±10.99
GNAL : 41.12±14.3

# 3. Count of Statistically Different Features Among Gene Pairs

In [143]:
mann_wu_results  = pd.read_csv("test_results/mann_whitney_u_results.csv")
fisher_e_results = pd.read_csv("test_results/fisher_e_results.csv")
feature_count    = len(mann_wu_results.biomarker.unique()) +  len(fisher_e_results.biomarker.unique())# both continious + binary features

In [147]:
df_differences   = pd.DataFrame(columns=["gene1","gene2","count", "percent"])

for gene1 in genes:
    for gene2 in genes:
        
        if(gene1!=gene2):
            mwu_diff       = len(mann_wu_results[(mann_wu_results.gene1==gene1) & (mann_wu_results.gene2==gene2) & (mann_wu_results.pvalue<=0.05)])
            fe_diff        = len(fisher_e_results[(fisher_e_results.gene1==gene1) & (fisher_e_results.gene2==gene2) & 
                                                  (fisher_e_results.test_type=="two-sided") & (fisher_e_results.pvalue<=0.05)])
            row            = {}
            row["gene1"]   = gene1
            row["gene2"]   = gene2
            row["count"]   = mwu_diff + fe_diff
            row["percent"] = (mwu_diff + fe_diff) / feature_count * 100
            df_differences.loc[len(df_differences)] = row 

In [149]:
# Create a new column 'gene_pair' by sorting 'gene1' and 'gene2' lexicographically
df_differences['gene_pair'] = df_differences[['gene1', 'gene2']].apply(lambda x: tuple(sorted(x)), axis=1)

# Drop duplicates based on the 'gene_pair' column
df_differences = df_differences.drop_duplicates(subset=['gene_pair'])
df_differences.reset_index(inplace=True, drop=True)

df_differences[df_differences.percent>=70]

,gene1,gene2,count,percent,gene_pair
3,AOPEP,PLA2G6,26,70.270270,"(AOPEP, PLA2G6)"
4,AOPEP,SGCE,27,72.972973,"(AOPEP, SGCE)"
9,GNAL,PANK2,28,75.675676,"(GNAL, PANK2)"
22,PANK2,SGCE,29,78.378378,"(PANK2, SGCE)"
27,PLA2G6,THAP1,30,81.081081,"(PLA2G6, THAP1)"
30,SGCE,THAP1,28,75.675676,"(SGCE, THAP1)"
32,SGCE,VPS16,27,72.972973,"(SGCE, VPS16)"


# 4. Percentage of Gene-Pairs Significant for Neural Features

In [190]:
df_significance   = pd.DataFrame(columns=["feature", "count", "percent", "medium_effet_size_count"])

for feature in mann_wu_results.biomarker.unique():
    dataset_feature              = pd.DataFrame(mann_wu_results[mann_wu_results.biomarker==feature])
    dataset_feature['gene_pair'] = dataset_feature[['gene1', 'gene2']].apply(lambda x: tuple(sorted(x)), axis=1)
    dataset_feature              = dataset_feature.drop_duplicates(subset=['gene_pair'])
    dataset_feature.reset_index(inplace=True, drop=True)
    
    row                            = {}
    row["feature"]                 = feature
    row["count"]                   = len(dataset_feature[dataset_feature.pvalue<=0.05])
    row["percent"]                 = len(dataset_feature[dataset_feature.pvalue<=0.05])/ len(dataset_feature) * 100
    row["medium_effet_size_count"] = len(dataset_feature[(dataset_feature.pvalue<=0.05) & (dataset_feature.cohens_d_interpretation!="small")])
    
    df_significance.loc[len(df_significance)] = row 

In [192]:
df_significance

,feature,count,percent,medium_effet_size_count
0,firing_rate,13,36.111111,4
1,firing_regularity,22,61.111111,18
2,cv,21,58.333333,12
3,lv,13,36.111111,6
4,isi_mean,13,36.111111,4
5,isi_std,18,50.000000,6
6,isi_skewness,22,61.111111,18
7,isi_rho,11,30.555556,4
8,pause_index,3,8.333333,0
9,pause_ratio,11,30.555556,3


In [182]:
df_significance[df_significance["count"]==22]

,feature,count,percent
1,firing_regularity,22,61.111111
6,isi_skewness,22,61.111111
